In [26]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      
                       'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'HW_0.78'#the classification target#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:13]
raw_power=raw_data.iloc[:,13]
print('ready')

Using matplotlib backend: Qt5Agg
ready


In [27]:
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    rank = [index for index, value in sorted(list(enumerate(best_model.feature_importances_)), key=lambda x:x[1],reverse=True)]
    name_list=[   
                       'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)'#13
    ]
    plt.bar(range(len(best_model.feature_importances_)), best_model.feature_importances_,color='rgb',tick_label=name_list)
    plt.xticks(rotation=90)
    plt.savefig('%s importance classification 0.78.png' %algorithm_name)

In [28]:
seed=117
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
#  'boosting_type':['gbdt','rf'],
#  'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
#  'n_estimators':[100,200,400,800],
#  'subsample':[0.5,0.6,0.7,0.8,0.9,1],
#  'reg_alpha':[0,0.001,0.01],
#  'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbose=0)
param_xg = {
#  'booster':['gbtree'],
#  'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
#  'n_estimators':[100,200,400,800],
#  'max_depth':[5,7,9,11],
#  'subsample':[0.5,0.6,0.7,0.8,0.9,1],
#  'reg_alpha':[0,0.001,0.01],
#  'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
#  'learning_rate':[0.01,0.05,0.1],
#  'n_estimators':[200,400],
#  'max_depth':[5,8,11],
#  'subsample':[0.5,0.6,0.7,0.8,0.9,1]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
#  'learning_rate':[0.01,0.05,0.1],
#  'n_estimators':[200,400],
#  'criterion':['friedman_mse','mae','mse'],
#  'max_features':['auto','sqrt','log2'],
#  'loss':['deviance', 'exponential']
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
#      'n_estimators':[100,200,400,800],
#      'criterion':['gini','entropy'],
#      'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
#         'max_depth':[5,7,9,11],
#         'criterion' : ['gini','entropy'],
#         'splitter' : [ "best",'random'],
#         'max_features':['auto','sqrt','log2']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
#         'max_depth':[5,7,9,11],
#         'criterion' : ['gini','entropy'],
#         'splitter' : [ "best",'random'],
#         'max_features':['auto','sqrt','log2']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
#      'n_estimators':[50,100,200,400,800],
#      'learning_rate':[0.01,0.02,0.05,0.1,0.2,1]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

start
Best Classifier: {} Best Score: 0.6495276653171389
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.93      0.59      0.72        22
           1       0.55      0.92      0.69        12

    accuracy                           0.71        34
   macro avg       0.74      0.75      0.70        34
weighted avg       0.79      0.71      0.71        34

LightGBM
[ 2 15 61 47  0 65 52 53 70 75 75 45 55 32]
start
Best Classifier: {} Best Score: 0.6126855600539811
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy   

In [ ]:
start
Best Classifier: {} Best Score: 0.6495276653171389
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.93      0.59      0.72        22
           1       0.55      0.92      0.69        12

    accuracy                           0.71        34
   macro avg       0.74      0.75      0.70        34
weighted avg       0.79      0.71      0.71        34

LightGBM
[ 2 15 61 47  0 65 52 53 70 75 75 45 55 32]
start
Best Classifier: {} Best Score: 0.6126855600539811
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.90      0.82      0.86        22
           1       0.71      0.83      0.77        12

    accuracy                           0.82        34
   macro avg       0.81      0.83      0.81        34
weighted avg       0.83      0.82      0.83        34

XGBoost
[0.01593158 0.01015779 0.01426379 0.02325905 0.0688468  0.02857259
 0.02248417 0.0234479  0.01458369 0.05869116 0.03456058 0.01686661
 0.02458215 0.6437521 ]
start
Best Classifier: {} Best Score: 0.6651821862348178
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       1.00      0.77      0.87        22
           1       0.71      1.00      0.83        12

    accuracy                           0.85        34
   macro avg       0.85      0.89      0.85        34
weighted avg       0.90      0.85      0.86        34

CatBoost
[ 2.04744795  2.15069424  7.00214855  6.62816314  0.28344346  9.5894701
  6.29374459  6.28437765  8.69625601 13.87229501  8.25631683  7.86478093
 10.73460096 10.29626058]
start
Best Classifier: {} Best Score: 0.6180836707152497
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.89      0.77      0.83        22
           1       0.67      0.83      0.74        12

    accuracy                           0.79        34
   macro avg       0.78      0.80      0.79        34
weighted avg       0.81      0.79      0.80        34

GradientBoost
[0.01420988 0.01261444 0.07406813 0.08760146 0.01100777 0.09654873
 0.05152259 0.07883405 0.10045109 0.16549194 0.09087264 0.03643463
 0.06700683 0.11333582]
start
Best Classifier: {} Best Score: 0.670310391363023
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.73      0.92      0.81        12

    accuracy                           0.85        34
   macro avg       0.84      0.87      0.85        34
weighted avg       0.87      0.85      0.86        34

Random Forest
[0.02529742 0.0214305  0.08533587 0.07076805 0.0118804  0.09037939
 0.07462562 0.07494754 0.10748617 0.11465019 0.08608846 0.07704311
 0.08979166 0.07027562]
start
Best Classifier: {} Best Score: 0.6385964912280702
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.88      0.68      0.77        22
           1       0.59      0.83      0.69        12

    accuracy                           0.74        34
   macro avg       0.74      0.76      0.73        34
weighted avg       0.78      0.74      0.74        34

Extra Tree
[0.00887504 0.0297205  0.08448129 0.08707802 0.05344151 0.07769753
 0.05604106 0.08178365 0.12924967 0.05078194 0.02663835 0.0719236
 0.15662119 0.08566664]
start
Best Classifier: {} Best Score: 0.6387314439946019
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       104
           1       1.00      1.00      1.00        87

    accuracy                           1.00       191
   macro avg       1.00      1.00      1.00       191
weighted avg       1.00      1.00      1.00       191

              precision    recall  f1-score   support

           0       0.78      0.64      0.70        22
           1       0.50      0.67      0.57        12

    accuracy                           0.65        34
   macro avg       0.64      0.65      0.64        34
weighted avg       0.68      0.65      0.65        34

Decision Tree
[0.         0.01876412 0.         0.06487467 0.04222552 0.12857256
 0.13374387 0.10562044 0.09833045 0.19646143 0.02884547 0.00370706
 0.0680827  0.1107717 ]
start
Best Classifier: {} Best Score: 0.6287449392712551
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       104
           1       0.91      0.89      0.90        87

    accuracy                           0.91       191
   macro avg       0.91      0.90      0.90       191
weighted avg       0.91      0.91      0.91       191

              precision    recall  f1-score   support

           0       0.86      0.82      0.84        22
           1       0.69      0.75      0.72        12

    accuracy                           0.79        34
   macro avg       0.77      0.78      0.78        34
weighted avg       0.80      0.79      0.80        34

AdaBoost
[0.04 0.04 0.1  0.08 0.   0.12 0.02 0.12 0.1  0.08 0.04 0.08 0.14 0.04]